# Máquinas de Vectores de Soporte (SVM)
## Presentación - Encuentro 4

---

## 1. ¿Qué es SVM?

### Concepto Simple
Las **Máquinas de Vectores de Soporte (SVM)** son algoritmos que encuentran la **mejor manera de separar** diferentes grupos de datos.

### Analogía Visual
Imagine que tiene **perros** y **gatos** mezclados en un patio. El SVM encuentra la **línea más amplia** entre ellos para separarlos claramente.

---

## 2. Los 4 Conceptos Fundamentales

### 2.1 Hiperplano
- **Qué es**: La línea (o plano) que separa las clases
- **Objetivo**: Buscar la línea que esté lo más lejos posible de ambos grupos
- **Visualización**:
  - 2D: una línea recta
  - 3D: un plano
  - nD: un hiperplano (generalización)

    ![](ml-svm-clasificador-c.png)

    [Imagen extraida de Luis G. Serrano - Machine Learning]()

### 2.2 Vectores de Soporte
- **Qué son**: Los puntos de datos **más cercanos** a la línea de separación
- **Importancia**: Solo necesitas estos puntos para clasificar nuevos datos
- **Ventaja**: Muy eficiente en memoria

  **Ejemplo**: 1000 muestras $\to$ 50 vectores (puntos) de soporte

### 2.3 Margen
- **Qué es**: El espacio libre entre la línea de separación y los puntos más cercanos
- **Objetivo**: Maximizar este espacio para crear una separación más robusta
- **Tipos**:
  - **Margen duro**: No permite errores
  - **Margen suave**: Permite algunos errores (más flexible)

    ![](ml-svm-clasificador-a.png)

    [Imagen extraida de Luis G. Serrano - Machine Learning]()

### 2.4 Kernel
- **Qué es**: Una función que permite separaciones **no lineales** (curvas)
- **Tipos principales**:
  - **Lineal**: Separaciones rectas (más rápido)
  - **RBF**: Separaciones curvas complejas (más flexible)
  - **Polinomial**: Separaciones con curvas polinómicas
- **Cuándo usar**: RBF para casos generales

---

## 3. Parámetros Principales

### Parámetro C (Regularización)
| Valor             | Significado                            | Uso Recomendado               |
|-------------------|----------------------------------------|-------------------------------|
| C grande (>100)   | Línea rígida, pocos errores permitidos | Cuando quieres alta precisión |
| C pequeño (<1)    | Línea flexible, más errores permitidos | Cuando los datos tienen ruido |
| **Valor inicial** | C=1                                    | Punto de partida estándar     |

### Parámetro gamma (Solo para kernel RBF)
| Valor                | Significado          | Uso Recomendado         |
|----------------------|----------------------|-------------------------|
| gamma grande (>1)    | Curvas muy complejas | Riesgo de sobreajuste   |
| gamma pequeño (<0.1) | Curvas suaves        | Mejor generalización    |
| **Valor inicial**    | gamma=0.1            | Valor seguro y estándar |

### Grid Search
- **Qué es**: Prueba automáticamente diferentes combinaciones de parámetros
- **Ventaja**: Encuentra la mejor combinación automáticamente
- **Resultado**: Te entrega el modelo ya optimizado

---

## 4. Flujo de Trabajo con SVM

### Paso a Paso

1. **Preparar datos**
   - Obtener los datos
   - Características y etiquetas
  
2. **Dividir datos**
   - 80% para entrenamiento
   - 20% para prueba

3. **Normalizar datos**
   - **CRÍTICO**: Normalizar con StandardScaler
   - SVM es muy sensible a la escala de los datos

4. **Grid Search**
   - Prueba diferentes combinaciones automáticamente
   - Ya entrena el modelo con los mejores parámetros

5. **Evaluar y visualizar**
   - Ver las fronteras de decisión
   - Calcular métricas de rendimiento

### Ejemplo: Comparación de SVM con GridSearchCV

In [ ]:
# SVM (lineal, RBF y polinomial), GridSearchCV, 5-Fold y PCA para visualización
# Incluye simulación de uso en producción y matriz de confusión en porcentajes

import matplotlib.pyplot as plt
import numpy as np
import warnings
import joblib
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
warnings.filterwarnings('ignore')

# Generar datos sintéticos
X, y = make_blobs(n_samples=1000, centers=4, n_features=10, cluster_std=5, random_state=42)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definir la grilla de hiperparámetros
param_grid = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10]},
    {'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': ['scale', 0.1, 1]},
    {'kernel': ['poly'], 'C': [0.1, 1, 10], 'degree': [2, 3, 4], 'gamma': ['scale', 0.1, 1]}
]

# Crear el modelo base y el GridSearchCV con validación cruzada (5 folds)
svm = SVC()
grid = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
grid.fit(X_train_scaled, y_train)

# Mostrar los mejores parámetros
print("Mejor combinación y puntuación encontrada por GridSearchCV:")
print(f"{grid.best_params_}, {grid.best_score_:.3f}")

# Evaluar el mejor modelo
best_svm = grid.best_estimator_
y_pred = best_svm.predict(X_test_scaled)

acc = accuracy_score(y_test, y_pred)
print(f"Precisión en el conjunto de prueba: {acc:.3f}")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Matriz de confusión en valores absolutos
cm = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusión (valores absolutos):")
print(cm)

# Matriz de confusión en porcentajes
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
print("\nMatriz de confusión (porcentajes):")
# Formato directo evitando notación científica
np.set_printoptions(precision=2, suppress=True, floatmode='fixed')
print(cm_percent)

graficar_matriz_de_confusion = False
if graficar_matriz_de_confusion:
    # Visualización de la matriz de confusión
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Matriz de confusión absoluta
    ConfusionMatrixDisplay.from_estimator(best_svm, X_test_scaled, y_test, cmap='Blues', ax=ax1)
    ax1.set_title("Matriz de confusión (valores absolutos)")
    ax1.set_xlabel("Predicción")
    ax1.set_ylabel("Valor real")

    # Matriz de confusión en porcentajes
    ConfusionMatrixDisplay(confusion_matrix=cm_percent, display_labels=best_svm.classes_).plot(ax=ax2, cmap='Blues', values_format='.2f')
    ax2.set_title("Matriz de confusión (porcentajes)")
    ax2.set_xlabel("Predicción")
    ax2.set_ylabel("Valor real")

    plt.tight_layout()
    plt.show()

graficar_fronteras_de_decision = False
if graficar_fronteras_de_decision:
    # Reducción a 2D con PCA (solo para visualización)
    if X_train_scaled.shape[1] > 2:
        print(f"\nAplicando PCA: reducción de {X_train_scaled.shape[1]} → 2 dimensiones para visualización.")
        pca = PCA(n_components=2)
        X_vis = pca.fit_transform(X_train_scaled)
    else:
        X_vis = X_train_scaled  # si ya tiene 2 dimensiones, no aplicamos PCA

    # Visualización de fronteras de decisión (no se usa para entrenar ni usar el modelo)
    plt.figure(figsize=(8,6))

    # Crear una malla 2D
    x_min, x_max = X_vis[:, 0].min() - 1, X_vis[:, 0].max() + 1
    y_min, y_max = X_vis[:, 1].min() - 1, X_vis[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                        np.linspace(y_min, y_max, 300))

    # Si los datos fueron reducidos con PCA, proyectamos la malla al espacio original para predecir
    if X_train_scaled.shape[1] > 2:
        # Invertimos la proyección PCA para volver al espacio escalado
        grid_points = np.c_[xx.ravel(), yy.ravel()]
        grid_points_scaled = pca.inverse_transform(grid_points)
        Z = best_svm.predict(grid_points_scaled)
    else:
        Z = best_svm.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)

    # Dibujar fronteras
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='Accent')
    plt.scatter(X_vis[:, 0], X_vis[:, 1], c=y_train, cmap='Accent', edgecolors='k')
    plt.title(f"Fronteras de decisión del mejor SVM ({grid.best_params_['kernel']})")
    plt.xlabel("Componente 1 (PCA)" if X_train_scaled.shape[1] > 2 else "Característica 1")
    plt.ylabel("Componente 2 (PCA)" if X_train_scaled.shape[1] > 2 else "Característica 2")
    plt.show()

# Guardar el modelo entrenado y el scaler para uso en producción
joblib.dump(best_svm, 'svm_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("\nModelo y scaler guardados para uso en producción:")
print("- svm_model.pkl")
print("- scaler.pkl")


## 5. ¿Cuándo Usar SVM?

### Ventajas
- **Eficiente en memoria**: Solo almacena vectores de soporte
- **Bueno con alta dimensionalidad**: Funciona bien con muchas características
- **Flexible**: Diferentes kernels para diferentes tipos de separación
- **Robusto**: Resiste datos atípicos

### Desventajas
- **Lento con datasets muy grandes** (>10,000 muestras)
- **Requiere normalización** (no opcional)
- **Parámetros sensibles** (requieren ajuste)

### Cuándo Usar
- Dataset pequeño a mediano (<10,000 muestras)
- Necesitas separaciones no lineales
- Trabajas con alta dimensionalidad
- Necesitas un modelo robusto

---

## 6. Métricas de Evaluación

### Matriz de Confusión

| Real \ Predicción | Positiva        | Negativa        |
|-------------------|-----------------|-----------------|
| **Positiva**      | **VP** o **TP** | FN              |
| **Negativa**      | FP              | **VN** o **TN** |

#### Ejemplo:

| Real \ Predicho | Perro       | Gato        | Pájaro      |
|-----------------|-------------|-------------|-------------|
| **Perro**       | **70 (TP)** | 5 (FN)      | 2 (FN)      |
| **Gato**        | 10 (FP)     | **85 (TP)** | 5           |
| **Pájaro**      | 0 (FP)      | 3           | **95 (TP)** |


### Métricas Principales

**Fórmulas básicas:**
- **Accuracy**: (TP + TN) / Total
- **Precision**: TP / (TP + FP)  
- **Recall**: TP / (TP + FN)
- **F1-Score**: 2 × (Precision × Recall) / (Precision + Recall)

| Métrica       | Interpretación                                |
|---------------|-----------------------------------------------|
| **Accuracy**  | Porcentaje de clasificaciones correctas      |
| **Precision** | De los predichos positivos, ¿cuántos son realmente positivos? |
| **Recall**    | De los realmente positivos, ¿cuántos detectamos? |
| **F1-Score**  | Balancea precisión y recall (media armónica)  |

### ¿Qué Métrica Usar?

| Escenario                    | Métrica Principal | Razón                           |
|------------------------------|-------------------|---------------------------------|
| Clases balanceadas           | **Accuracy**      | Representa bien el rendimiento  |
| Clase minoritaria importante | **F1-Score**      | Balancea ambas métricas         |
| Evitar falsas alarmas        | **Precision**     | Ej: spam vs no-spam             |
| Detectar todos los casos     | **Recall**        | Ej: detección de fallas, cáncer |
| Clases desbalanceadas        | **Macro F1**      | No se sesga por mayoría         |

---

## 7. Mejores Prácticas

### Hacer SIEMPRE
1. **Normalizar datos** (crítico)
2. Usar **Grid Search** para encontrar mejores parámetros
3. **Validación cruzada** para evaluar robustez
4. Comparar **train vs test accuracy** para detectar sobreajuste
5. Empezar con valores conservadores de C y gamma

### Errores Comunes
1. **No normalizar** datos de prueba correctamente
2. **Usar datos de prueba durante entrenamiento**
3. **Ignorar sobreajuste** (train accuracy alta, test baja)
4. **Valores extremos** de C y gamma sin validar
5. **No visualizar** las fronteras de decisión

### Interpretación de Resultados
- **Pocos vectores de soporte**: Modelo simple y generalizable
- **Muchos vectores de soporte**: Modelo complejo (posible sobreajuste)
- **Train accuracy alta, test baja**: Sobreajuste (reducir C o gamma)
- **Baja accuracy en ambos**: Bajo rendimiento (cambiar kernel o parámetros)

---

## 8. El Camino de los Datos: Entrenamiento → Predicción

  ![](ml-svm-entren-pred.svg)

---

## 9. Entrenamiento del modelo

  ![](ml-svm-entren-modelo.svg)

In [ ]:
import numpy as np
import joblib

# Función para inspeccionar el modelo cargado
def inspect_model():
    """
    Inspecciona el modelo SVM cargado y muestra información detallada
    """
    print("\n" + "="*50)
    print("INSPECCIÓN DEL MODELO SVM")
    print("="*50)

    # Cargar el modelo
    model = joblib.load('svm_model.pkl')

    # Información básica del modelo
    print(f"Parámetros del modelo  {type(model).__name__}:")
    print(f"  - Kernel: {model.kernel}")
    print(f"  - C: {model.C}")
    print(f"  - Gamma: {model.gamma}")

    # Información del entrenamiento
    print(f"\nInformación del entrenamiento:")
    print(f"  - Número de clases y clases:     {len(model.classes_)} - {model.classes_}")
    print(f"  - Número de características:     {model.n_features_in_}")
    print(f"  - Número de vectores de soporte: {model.n_support_}")
    print(f"  - Vectores de soporte por clase: {model.n_support_}")

    return model

# Función para simular predicciones en producción
def predict_production_samples(new_samples):
    """
    Simula el uso del modelo en producción con nuevas muestras

    Args:
        new_samples: array o DataFrame con las nuevas muestras a predecir

    Returns:
        predictions: array con las predicciones
        probabilities: array con las probabilidades de cada clase
    """
    # Cargar el modelo y scaler guardados
    model = joblib.load('svm_model.pkl')
    scaler_loaded = joblib.load('scaler.pkl')

    # Normalizar las nuevas muestras
    new_samples_scaled = scaler_loaded.transform(new_samples)

    # Hacer predicciones
    predictions = model.predict(new_samples_scaled)

    # Obtener probabilidades (si el modelo las soporta)
    try:
        probabilities = model.predict_proba(new_samples_scaled)
    except AttributeError:
        # Para SVM sin probabilidades, usar decision_function
        decision_scores = model.decision_function(new_samples_scaled)
        probabilities = decision_scores

    return predictions, probabilities

# Inspeccionar el modelo
model = inspect_model()

# Simulación de uso en producción
print("\n" + "="*50)
print("SIMULACIÓN DE USO EN PRODUCCIÓN")
print("="*50)

# Generar nuevas muestras sintéticas para simular datos de producción
# np.random.seed(123)
n_samples = 20
n_features = model.n_features_in_
new_samples = np.random.randn(n_samples, n_features) * 3 + np.random.randn(n_samples, n_features) * 2

print(f"\nPrediciendo {len(new_samples)} nuevas muestras con {n_features} características cada una...")
predictions, probabilities = predict_production_samples(new_samples)

# Mostrar resultados
print("\nResultados de las predicciones:")
for i, (pred, prob) in enumerate(zip(predictions, probabilities)):
    if len(prob.shape) > 1:  # Si tenemos probabilidades por clase
        prob_str = f"Probabilidades: {prob.round(3)}"
    else:                    # Si no tenemos probabilidades, tenemos decision scores
        prob_str = f"Decision scores: {prob.round(3)}"

    print(f"Muestra {i+1:2d}: Clase predicha = {pred}, {prob_str}")

# Estadísticas de las predicciones
unique, counts = np.unique(predictions, return_counts=True)
print(f"\nDistribución de predicciones:")
for cls, count in zip(unique, counts):
    percentage = (count / len(predictions)) * 100
    print(f"Clase {cls}: {count} muestras ({percentage:.1f}%)")

print("\nSimulación completada exitosamente!")

## 10. Puntos Clave para Recordar

### Los 3 Conceptos Esenciales
1. **Hiperplano**: La línea que separa clases
2. **Vectores de Soporte**: Puntos clave que definen la separación
3. **Margen**: Espacio que queremos maximizar

### Los 2 Parámetros Críticos
1. **C**: Controla cuánto errores permitir (C grande = menos errores)
2. **gamma**: Controla complejidad de curvas (gamma pequeño = más simple)

### El Flujo Clave
1. Normalizar (SIEMPRE)
2. Grid Search (optimización automática)
3. Visualizar (entender el modelo)
4. Evaluar (métricas adecuadas)

---

## 11. Próximos Pasos

### Para Practicar
1. Ejecutar el notebook de práctica
2. Probar diferentes kernels (lineal, RBF)
3. Cambiar parámetros C y gamma
4. Aplicar a dataset real (ej: Iris)
5. Experimentar con datos multiclase

### Para Profundizar
- Explorar SVM multiclase
- Implementar en casos industriales reales
- Comparar con otros algoritmos (Random Forest, Redes Neuronales)
- Estudiar teoría matemática de SVM

---

## 12. Recursos y Referencias

### Documentación
- Grokking Machine Learning, Luis G. Serrano, 2021
- [Scikit-learn: SVM](https://scikit-learn.org/stable/modules/svm.html)
- [Grid Search Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

### Para Aprender Más
- Libro: "Pattern Recognition and Machine Learning" - Christopher Bishop
- Curso: Machine Learning por Andrew Ng (Coursera)

---

## ¿Preguntas?

*Tiempo restante para preguntas y discusión*

---

## Apéndice: Comparación Rápida con Otros Algoritmos

| Algoritmo           | Interpretabilidad | Escalabilidad | Datos No Lineales | Velocidad |
|---------------------|-------------------|---------------|-------------------|-----------|
| **SVM**             | Media             | Baja          | Excelente         | Media     |
| Random Forest       | Alta              | Alta          | Buena             | Alta      |
| Redes Neuronales    | Baja              | Media         | Excelente         | Media     |
| Regresión Logística | Alta              | Alta          | Limitada          | Alta      |

